# 5-Day AI Agents Intensive: Assignment 1
## Introduction to AI Agents and Multi-Agent Architectures

---

**Course Overview**

This assignment combines theoretical understanding with practical implementation of AI agents using Google's Agent Development Kit (ADK). You'll learn:

- ✅ Core concepts of AI agents and agentic systems
- ✅ Building single agents with tools
- ✅ Designing multi-agent architectures
- ✅ Implementing workflow patterns (Sequential, Parallel, Loop)

---

## Part 1: Theoretical Foundation

### 1.1 From Predictive AI to Autonomous Agents

Artificial intelligence is undergoing a paradigm shift. Traditional AI models excel at passive, discrete tasks like answering questions or generating images, but they require constant human direction. We're now moving toward **AI agents** - complete applications capable of autonomous problem-solving and task execution.

**Key Distinction:**
- **Traditional AI**: `Prompt → LLM → Text`
- **AI Agent**: `Prompt → Agent → Think → Act → Observe → Final Answer`

### 1.2 What is an AI Agent?

An AI agent combines four essential components:

1. **The Model (The "Brain")**: Core reasoning engine (LM/foundation model) that processes information and makes decisions
2. **Tools (The "Hands")**: Mechanisms connecting reasoning to the real world (APIs, functions, databases)
3. **Orchestration Layer (The "Nervous System")**: Governs the operational loop, manages planning, memory, and reasoning strategies
4. **Deployment (The "Body and Legs")**: Production infrastructure making the agent accessible and reliable

### 1.3 The Agentic Problem-Solving Process

Agents operate on a **five-step cyclical process**:

1. **Get the Mission**: Receive a specific, high-level goal
2. **Scan the Scene**: Gather context from available resources
3. **Think It Through**: Devise a plan using the reasoning model
4. **Take Action**: Execute the plan using appropriate tools
5. **Observe and Iterate**: Analyze outcomes and repeat until goal is achieved

**Example: Customer Support Agent**

User asks: "Where is my order #12345?"

The agent:
1. Plans a multi-step strategy (find order → track shipment → report)
2. Calls `find_order("12345")` tool
3. Observes: order found with tracking number "ZYX987"
4. Calls `get_shipping_status("ZYX987")` tool
5. Observes: "Out for Delivery"
6. Generates response: "Your order #12345 is 'Out for Delivery'!"

### 1.4 Taxonomy of Agentic Systems

**Level 0: Core Reasoning System**
- LM operates in isolation with pre-trained knowledge
- No tools, memory, or real-time data access
- Can explain concepts but cannot access current information

**Level 1: Connected Problem-Solver**
- Connects to external tools (search APIs, databases)
- Can retrieve real-time information
- Example: Searching for current stock prices or weather

**Level 2: Strategic Problem-Solver**
- Plans complex, multi-step tasks
- Employs context engineering to manage information
- Example: Finding coffee shops midway between two locations

**Level 3: Collaborative Multi-Agent System**
- Team of specialized agents working together
- Agents treat other agents as tools
- Example: Project manager delegating to research, marketing, and web dev agents

**Level 4: Self-Evolving System**
- Identifies capability gaps and creates new tools/agents
- Dynamically expands its own capabilities
- Represents the frontier of autonomous systems

### 1.5 Multi-Agent Design Patterns

When tasks grow complex, specialized agent teams become more effective than single "super-agents":

**Coordinator Pattern**: Manager agent routes sub-tasks to specialists

**Sequential Pattern**: Linear workflow where output of one agent becomes input for next

**Iterative Refinement Pattern**: Generator creates content, critic evaluates, loop continues

**Human-in-the-Loop (HITL) Pattern**: Agent pauses for human approval at critical decisions

---

## Part 2: Practical Implementation

### Setup and Configuration

We'll use Google's Agent Development Kit (ADK) to build our agents. ADK provides:
- Modular framework for agent development
- Optimized for Gemini models
- Support for Python, Java, and Go
- Built-in tools and orchestration capabilities

---

### ⚙️ Section 1: Environment Setup

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [4]:
# %pip install google-generativeai 
# %pip install google-adk kaggle nbformat 

#### 1.1 Configure Gemini API Key

**Instructions:**
1. Get your API key from [Google AI Studio](https://aistudio.google.com/app/api-keys)
2. In Kaggle: Add-ons → Secrets → Create `GOOGLE_API_KEY`
3. Ensure the checkbox is selected to attach the secret

In [ ]:
# import os
# from kaggle_secrets import UserSecretsClient

# try:
#     GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
#     os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#     print("✅ Gemini API key setup complete.")
# except Exception as e:
#     print(f"🔒 Authentication Error: {e}")
#     print("Please add 'GOOGLE_API_KEY' to your Kaggle secrets.")

import os

# 🔑 Replace with your actual Gemini API key from Google AI Studio
os.environ["GOOGLE_API_KEY"] = ""   #YOUR_API_KEY_HERE

if "GOOGLE_API_KEY" in os.environ:
    print("✅ Gemini API key setup complete.")
else:
    print("❌ API key not found.")


#### 1.2 Import ADK Components

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

#### 1.3 Configure Retry Options

Handle transient errors like rate limits with exponential backoff:

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,  # Initial delay before first retry (seconds)
    http_status_codes=[429, 500, 503, 504]  # Retry on these HTTP errors
)

print("✅ Retry configuration set.")

---

## Task 1: Building Your First AI Agent

### Objective
Create a single agent that can search for current information and provide informed responses.

### What You'll Learn
- How to configure an agent with a model, instructions, and tools
- How agents use tools to access real-time information
- The difference between static LM responses and agent-powered responses

---

### Step 1: Define Your First Agent

We'll create an agent with:
- **Model**: Gemini 2.5 Flash Lite (fast, efficient)
- **Instruction**: Clear guidance on when to use tools
- **Tools**: Google Search for current information

In [ ]:
# Create a helpful assistant agent with Google Search capability
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

### Step 2: Run Your Agent

Create a runner and send a query that requires current information:

In [ ]:
# Create a runner for the agent
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner created.")

# Ask about ADK - requires current information
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)

### Step 3: Try Your Own Query

Test the agent with a question requiring current information:

In [ ]:
# Try asking about current events or information
response = await runner.run_debug("What's the weather in London?")

# Feel free to modify the query above and try:
# - "Who won the last soccer world cup?"
# - "What new movies are showing in theaters now?"
# - Your own question!

### 🎯 Task 1 Reflection

**Key Takeaways:**
1. The agent didn't just respond - it **reasoned** that it needed more information
2. It **acted** by using the Google Search tool
3. It **observed** the results and formulated an answer
4. This ability to take action is the foundation of agent-based AI

**What's Next?**
A single agent is powerful, but complex tasks require teams. Let's build multi-agent systems!

---

## Task 2: Multi-Agent Systems & Workflow Patterns

### Objective
Build specialized agent teams that collaborate using different workflow patterns.

### The Problem with Monolithic Agents
Single "do-it-all" agents become:
- Hard to debug (which part failed?)
- Difficult to maintain (long, complex instructions)
- Unreliable (trying to do too much)

### The Solution: Team of Specialists
Multiple simple agents, each with one clear job, collaborating like a real team.

---

### Pattern 1: LLM-Based Orchestration

Let an LLM coordinator decide which specialist agents to call and when.

**Example: Research & Summarization System**

In [ ]:
# Research Agent: Searches for information
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic 
    and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",
)

# Summarizer Agent: Creates concise summaries
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

# Root Coordinator: Orchestrates the workflow
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information.
2. Next, after receiving findings, you MUST call the `SummarizerAgent` tool.
3. Finally, present the final summary clearly to the user.""",
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ LLM-orchestrated multi-agent system created.")

In [ ]:
# Run the multi-agent system
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)

### Pattern 2: Sequential Workflow - The Assembly Line

**Use Case**: When tasks must happen in a specific, guaranteed order.

**Example: Blog Post Creation Pipeline**

Outline Agent → Writer Agent → Editor Agent

In [ ]:
# Outline Agent: Creates blog structure
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",
)

# Writer Agent: Writes full blog post
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",
)

# Editor Agent: Polishes the draft
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Edit this draft: {blog_draft}
    Polish by fixing grammatical errors, improving flow and sentence structure, 
    and enhancing overall clarity.""",
    output_key="final_blog",
)

# Sequential Agent: Runs agents in fixed order
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

In [ ]:
# Run the sequential pipeline
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)

### Pattern 3: Parallel Workflow - Independent Researchers

**Use Case**: When tasks are independent and can run simultaneously for speed.

**Example: Multi-Topic Research**

Tech Research | Health Research | Finance Research → Aggregator

In [ ]:
# Create three specialized researchers
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",
)

health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",
)

finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",
)

# Aggregator combines all research
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Combine these three research findings into a single executive summary:

 **Technology Trends:** {tech_research}
 **Health Breakthroughs:** {health_research}
 **Finance Innovations:** {finance_research}
 
 Highlight common themes, surprising connections, and key takeaways. 
 Final summary should be around 200 words.""",
    output_key="executive_summary",
)

# Parallel execution then sequential aggregation
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

In [ ]:
# Run parallel research with aggregation
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)

### Pattern 4: Loop Workflow - The Refinement Cycle

**Use Case**: When output needs iterative improvement through feedback cycles.

**Example: Story Writing & Critique Loop**

Writer → Critic → Refiner → (repeat until approved)

In [ ]:
# Initial Writer: Creates first draft
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Based on the user's prompt, write the first draft of a short story 
    (around 100-150 words). Output only the story text, with no introduction or explanation.""",
    output_key="current_story",
)

# Critic: Provides feedback or approval
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a constructive story critic. Review the story: {current_story}
    
    Evaluate plot, characters, and pacing.
    - If the story is well-written and complete, respond EXACTLY: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",
)

# Exit function for loop termination
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating 
    the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

# Refiner: Improves story or exits loop
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a story refiner. You have:
    Story Draft: {current_story}
    Critique: {critique}
    
    Analyze the critique:
    - IF critique is EXACTLY "APPROVED", call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story to fully incorporate the feedback.""",
    output_key="current_story",
    tools=[FunctionTool(exit_loop)],
)

# Loop for refinement cycles
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# Root Sequential Agent: Initial Write → Refinement Loop
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

In [ ]:
# Run the story refinement system
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)

### 🎯 Task 2 Reflection: Choosing the Right Pattern

**Decision Tree: Which Workflow Pattern?**

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based Orchestration** | Dynamic decisions needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |

**Key Takeaways:**
- Multi-agent systems scale better than monolithic agents
- Choose patterns based on task requirements (order, independence, iteration)
- Use `output_key` for state passing between agents
- Always include safeguards like `max_iterations` in loops

---

## ✅ Congratulations!

You've built single and multi-agent systems! You've seen how agents reason, act, and collaborate.

**ℹ️ Note: No submission required!**
This is for hands-on practice. No need to submit to complete the course.

### 📚 Learn More

- [ADK Documentation](https://google.github.io/adk-docs/)
- [Agents in ADK](https://google.github.io/adk-docs/agents/)
- [Sequential Agents](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)
- [Parallel Agents](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/)
- [Loop Agents](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/)

### 🎯 Next Steps
Ready for Day 2? Explore custom tools and long-running operations!